# Converting from decimal to binary and back

## 0 Representations of a number in different bases

It is important to differentiate a real number $ x $ from the its representation in some system (binary, decimal, octal, hexadecimal, etc.). Any *integer* $ b \ge 2 $ can serve as the **base** of such a system. Once such a choice is made, $ x $ can be expressed in the form

$$ x = (a_m\, a_{m - 1}\dots a_1\, a_0\,.\,a_{-1}\, a_{-2} \dots)_{b}\ , $$

for some integers $ a_k $ satisfying $ 0 \le a_k \le b - 1 $, which are called the **digits** of $ x $ in base $ b $. This notation means that

$$ x = a_mb^m + a_{m - 1}b^{m - 1} + \dots + a_1 b + a_0 
   + a_{-1}b^{-1} + a_{-2}b^{-2} + \dots. $$
   
The number of digits to the *left* of the point is always *finite*, because any real number is bounded. In contrast, the number of digits to the *right* of the point can be *infinite*, since not every number is rational.

**Exercise:** Let $ b \ge 2 $ be an integer. Prove that the representation of a real number $ x $ is finite in base $ b $ if and only if $ x $ can be expressed as a fraction in which the numerator is an integer and the denominator is of the form $ b^r $ for some integer $ r \ge 0 $. 

⚠️ Because we use the decimal system almost exclusively to represent numbers in daily life, we write, say, $ x = 592 $ instead of $ x = (592)_{10} $. However, it is important to keep in mind that '$ 592 $' is not the number $ x $ itself, but rather its *representation* in base $ 10 $. The same number could also be written as $ x = (1001010000)_2 $, and were $ b = 2 $ the most commonly used base, we would instead abuse our notation to write $ x = 1001010000 $.

📝 The same real number $ x $ has infinitely many representations, one for each possible base $ b \ge 2 $.

**Exercise:** Determine the number $ (1001101)_2 $.

**Exercise:** Determine the number $ (210210)_3 $.

Most computing uses the **binary** form of representing numbers, in which the base $ b $ equals $ 2 $ and there are only two possible digits: $ 0 $ and $ 1 $. Such a digit is usually referred to as a **bit** (short for *binary digit*). There are several reasons why the binary system is prevalent. One is that it is the simplest one; another is the close relationship between Boolean logic and the arithmetic of integers modulo 2, given by the correspondence below:

| Boolean expression/operation | Binary equivalent |
| :----------------- | :----------------- |
| False              | 0                 |
| True               | 1                 |
| and                | $ \times $ (multiplication) |
| or                 | $ + $ (addition)  |
| not                | $ x \mapsto 1 + x $ |

Note that all three operations on the right side are *modulo 2*; for instance, $ 1 + 1 \pmod 2 = 0 \pmod 2 $.

Our main objective is to understand how to efficiently convert from the binary system to the more familiar decimal system and back.

For example, 

\begin{align} (1001.101)_2 &= 1 \times 2^3 + 0 \times 2^2 + 0 \times 2^1 + 1 \times 2^0 + 1 \times 2^{-1} + 0 \times 2^{-2} + 1 \times 2^{-3}  \\
& = 8 + 1 + \frac{1}{2} + \frac{1}{8} \\ &= \frac{77}{8}.
\end{align}

However, since there is no additional difficulty, and much to be gained, in working with an arbitrary base $ b \ge 2 $, we will formulate our results and scripts to cover any base.


📝 Besides the binary and decimal systems, the following two systems are also relatively important in applications:
* The *octal* system, where $ b = 8 $ and the possible digits are $ 0,1,\dots,7 $;
* The *hexadecimal* system, where $ b = 16 $ and the possible digits are $ 0, 1, \dots, 9 $ and $ A, B, \dots, F $, which stand for $ 10, 11, \dots, 15 $ in the decimal system, respectively.

**Exercise:** Determine the number $ (ABC)_{16} $.

**Exercise:** Show that if an integer $ n $ has a binary representation of length $ m $, then its representation in the octal system is of length at most $ \lceil\frac{n}{3}\rceil $.

## 1 Computing a number from its representation in base $ b $<a name="section2"></a>

In this section we will consider the following:

<a name="Problem1"></a>**Problem 1 (representation to number):** Given a base $ b $ and the representation of an unknown real number $ x $ in this base, determine $ x $.

📝 Even though the problem has been formulated in full generality, in practice we can only determine $ x $ precisely when the given representation is *finite*. Otherwise, we need to content ourselves with an approximation to $ x $.

<div class="alert alert-warning"> The following observation will be very useful: Any real number $ x $ can be written as the sum of a unique <i>integer</i> $ n $ and a unique <i>fractional number</i> $ t $ such that $ 0 \le t < 1 $. In Python, they are given by:
<ul>
    <li> $ t = $ <code>x % 1</code>;</li>
    <li> $ n = $ <code>int(x)</code>.</li>
    </ul>

We will call $ n $ the <b>integral part</b> of $ x $ and $ t $ its <b>fractional part</b>
</div>

📝 The function `modf` from the **math** module returns the tuple $ (t, n) $ when applied to $ x $.

By means of the aforementioned decomposition, we can reduce Problem 1 to two easier problems:

**Subproblem 1.1:** Given a base $ b $ and the representation of an unknown *integer* $ n $ in this base, determine $ n $.

**Subproblem 1.2:** Given a base $ b $ and the representation of an unknown *fractional number* $ 0 \le t < 1 $ in this base, determine $ t $.

**Example:**

In [24]:
def decompose(x: float) -> tuple:
    print(f"The decomposition of {x} into its integral and fractional part is:")
    print(f"{x} = {int(x)} + {x % 1}\n")


x = 3.14159
y = -1.4
z = -34.9

decompose(x)
decompose(y)
decompose(z)

The decomposition of 3.14159 into its integer and fractional part is:
3.14159 = 3 + 0.14158999999999988

The decomposition of -1.4 into its integer and fractional part is:
-1.4 = -1 + 0.6000000000000001

The decomposition of -34.9 into its integer and fractional part is:
-34.9 = -34 + 0.10000000000000142



In [27]:
def rep_to_number(b: int, digits: str) -> int:   # The type annotations are optional!
    """A function which takes a string of digits between 0 and b - 1,
    where the base b >= 1 is an integer, and returns the integer
    whose representation in base b is the given one."""
    
    assert isinstance(b, int) and b >= 2         # Make sure b is an integer >= 2.
    
    allowed = [str(d) for d in range(0, b)]      # Allowed digits.
    for digit in digits:                         # Check if the representation is valid.
        if digit not in allowed:
            raise ValueError(f"Invalid representation of a number in base {b}.")
       
    highest_exponent = len(digits) - 1
    n = 0
    exponent = highest_exponent
    for digit in digits:
        n += int(digit) * b**exponent
        exponent -= 1
    
    return n


print(rep_to_number(2, "1011"))
print(rep_to_number(3, "1011"))
print(rep_to_number(10, "1011"))

11
31
1011


### 2.2 Obtaining a fractional number from its representation in base $ b $
**Problem:** Define a function which takes a base $ b $ and the (finite) *representation* of a number $ t $ ($ 0 \le t < 1 $) in this base, given as a string of digits between $ 0 $ and $ b - 1 $, and returns $ t $ as output.

*Solution:* We can easily solve this by making use of the function defined in the preceding section as follows. Since $ 0 \le t < 1 $ by hypothesis, its representation in base $ b $ must be of the form
$$ t = (.a_{-1}\,a_{-2}\dots a_{-m})_b $$
for some $ m \ge 1 $. Equivalently,
$$ t = a_{-1}b^{-1} + a_{-2}b^{-2} + \dots + a_{-m}b^{-m} .$$
Multiplying both sides by $ b^ m $:
$$ b^m t = a_{-1}b^{m - 1} + a_{-2}b^{m - 2} + \dots + a_{-m} b^0 .$$
In other words, the *integer* $b^m t $ has the representation
$$ b^m t = (a_{-1}\,a_{-2}\dots a_{-m})_b .$$
(Note the absence of the point in the latter representation.) Therefore, we can obtain the value of $ t $ by first computing the *integer* $ b^m t $ using the preceding representation and then multiplying the result by $ b^{-m} $.

In [2]:
def rep_to_fractional(b: int, digits: str) -> int:   # The type annotations are optional!
    """A function which takes a string of digits between 0 and b - 1,
    where the base b >= 1 is an integer, and returns the fractional number
    whose representation in base b is the given one."""
    m = len(digits)
    n = rep_to_number(b, digits)
    return n * b**(-m)


print(rep_to_fractional(2, "11"))
print(rep_to_fractional(3, "11"))
print(rep_to_fractional(10, "11"))

0.75
0.4444444444444444
0.11


### 2.3 Obtaining a real number from its representation in base $ b $

**Exercise:** Define a function which takes three arguments:
* The base $ b $;
* The representation of the integral part $ n \ge 1 $ of a real number $ x $ in base $ b $;
* The representation of the fractional part $ 0 \le t < 1 $ of a real number $ x $ in base $ b $;

and which returns $ x $ as output. (*Hint:* Use the functions that were defined above.)

**Exercise:** How should each of the functions above be modified to include the possibility that $ x \le 0 $?

## 2 Computing the representation of a number in base $ b $

We will now consider the converse to [Problem 1](#Problem1), namely, how to compute the representation of a number in base $ b $ given the number $ x $ itself.

<a name="Problem2"></a>**Problem 2 (number to representation):** Given a base $ b $ and a real number $ x $, find its representation in base $ b $.

**Subproblem 2.1:** Given a base $ b $ and an *integer* $ n $, find its representation in base $ b $.

**Subproblem 2.2:** Given a base $ b $ and a *fractional number* $ t $, $ 0 \le t < 1 $, find its representation in base $ b $.

As before, if we can solve these two subproblems, then we can obtain a solution to Problem 2 by combinining their answers.


### 2.1 Obtaining the representation in base $ b $ of an integer

**Solution to Subproblem 2.1:** We may assume without loss of generality that $ n \ge 0 $, since the representation of $ -n $ is identical to that of $ n $ except for the $ - $ sign at the beginning. We are given the representation
$$ n = (a_m\,a_{m-1}\dots a_1a_0)_b\ ,$$
that is, 
$$ n = a_m\, b^m + a_{m-1}\, b^{m-1} + \dots + a_1 \, b^1 + a_0\, b^0 .$$
Now, upon (integer) division of $ n $ by $ b $, we obtain:
* $ a_0 $ as the remainder;
* $ a_m\, b^{m - 1} + a_{m - 1}\, b^{m - 2} + \dots + a_2\,b^1 + a_1 \, b^0 $ as the quotient.

Similarly, division of the latter by $ b $ yields:
* $ a_1 $ as the remainder;
* $ a_m\, b^{m - 2} + a_{m - 1}\, b^{m - 3} + \dots + a_3\,b^1 + a_2 \, b^0 $ as the quotient.

Clearly (or, more formally, by induction), continuing in this way *we can obtain the digits $ a_0,\, a_1, \dots, a_m $ as the successive remainders of division of $ n $ by $ b $*, which solves the problem. $ \square $

This solution is implemented in the following script:

In [31]:
def integer_to_rep(b: int, n: int) -> str:
    """Given a base b and a nonnegative integer n, returns the
    representation of n in base b as a string of digits from 0 to (b - 1)."""
    assert isinstance(b, int) and b >= 2
    assert isinstance(n, int) and n >= 0
    
    list_of_digits = []                   # Will store the list of digits of n.
    while n > 0: 
        d = n % b
        n //= b
        list_of_digits.append(str(d))     # Convert d to a string and append it.

    # Now reverse the list of digits and convert it
    # into a string by joining its elements to the empty string:
    list_of_digits.reverse()
    representation = ''.join(list_of_digits)
    return representation


print(integer_to_rep(2, 63))
print(integer_to_rep(2, 64))

111111
1000000


### 3.1 Obtaining a the representation in base $ b $ of a fractional number

**Solution to Subproblem 2.2:** Let 
$$ t = (.a_{-1}\,a_{-2}\dots a_{-r}\dots)_b $$
be this representation, that is,
$$ t = a_{-1}b^{-1} + a_{-2}b^{-2} + \dots + a_{-r}b^{-r} + \dots .$$
Multiplying both sides by $ b $, we deduce that
$$ bt = a_{-1} + \big( a_{-2}b^{-1} + \dots + a_{-r}b^{-r + 1} + \dots \big) .$$

Since our computer (and minds) has limited memory, we can only compute finitely many of the digits $ a_k $. To determine, say, the digits $ a_{-1} $ through $ a_{-r} $, begin by multiplying both sides of the latter equation by $ b^m $ to obtain:
$$ b^rt = a_{-1}\ b^{r - 1} + a_{-2}\ b^{r - 2} + \dots + a_{-r + 1}\ b^{1} + a_{-r} +
\big(a_{-r - 1}\  b^{-1} + a_{-r - 2}\ b^{-2} + \dots \big) .$$
Note that the term in parentheses is the fractional part of $ b^rt $. Applying our solution to Subproblem 2.1 to the integral part of $ b^mt $ gives the digits $ a_{-1} $ through $ a_{-r} $. $ \square $

This solution is implemented in the script below.

In [40]:
def fractional_to_rep(b: int, t: float, r: int) -> str:
    """Given a base b, a fractional number t and a positive integer r,
    returns the first r digits of the representation of t in base b
    as a string of digits from 0 to (b - 1)."""
    assert isinstance(b, int) and b >= 2
    assert isinstance(t, float)
    assert isinstance(r, int) and r >= 1
    
    n = int(b**r * t)
    representation = '.' + integer_to_rep(b, n)
    return representation


print(fractional_to_rep(2, 1 / 8, 10))
print(fractional_to_rep(10, 1 / 3, 10))


.10000000
.3333333333


In [54]:
def number_to_rep(b: int, x: float, r: int) -> str:
    """Given a base b, a fractional number t and a positive integer r,
    returns the representation of x in base b up to the r-th digit after
    the decimal point, as a string of digits from 0 to (b - 1)."""
    assert isinstance(b, int) and b >= 2
    assert isinstance(x, float)
    assert isinstance(r, int) and r >= 1
    
    rep_integral_part = integer_to_rep(b, int(x))
    rep_fractional_part = fractional_to_rep(b, x % 1, r)
    representation = rep_integral_part + rep_fractional_part
    return representation


print(number_to_rep(2, 15 / 8, 5))
print(number_to_rep(10, 7 / 3, 10))

1.11100
2.3333333333


We saw in the preceding section how to compute a real number given its representation in any base $ b \ge 2 $. We would now like to consider the problem of converting from the binary to the decimal representation (and back) directly.

**Problem:** Let $ (a_m\,a_{m-1}\dots a_1\,a_0)_2 $ be the 

## 3 Python built-in functions for conversion between bases

📝 Python provides the built-in functions

* `bin`
* `oct`
* `hex`

which take a single *integer* as argument and return its binary, octal or hexadecimal representation, respectively, in the form of strings.

**Example:** 

In [61]:
x = 12
y = 63
z = -15

print(bin(x), oct(x), hex(x))
print(bin(y), oct(y), hex(y))
print(bin(z), oct(z), hex(z))

0b1100 0o14 0xc
0b111111 0o77 0x3f
-0b1111 -0o17 -0xf


📝 Note how each string is prepended by `0b`, `0o` or `0x`, respectively, to indicate the type of the representation. Note also that the hexadecimal digits $ A, \dots, F $ appear in lowercase.